In [ ]:
import control as ctrl
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import sympy as sym
import ipywidgets as widgets
import IPython as IPy
import pandas as pd

In [ ]:
#################################
# Second-order All-Pole Systems #
#################################

# Symbolic variables
#   k -> DC gain
#   d -> damping ratio
#   w -> Natural frequency
#   H -> Transfer function
k,d,w,H=sym.symbols('k d w H(s)', real=True)

# Imaginary number symbol
s = sym.Symbol('s')

# Creating and displaying the symbolic transfer function
tf = sym.Eq(H, (k * w**2) / (s**2 + 2 * d * w * s + w**2))
tf

In [ ]:
# Importance of damping ratio value

# Define the transfer function with initial damping ratio
zeta_initial = 0.7  # You can change this value initially

w = 1
k = 1

num = [k * w**2]
den = [1, 2*w*zeta_initial, w**2]  # Example transfer function

sys = ctrl.TransferFunction(num, den)

# Create a slider for damping ratio
zeta_slider = widgets.FloatSlider(value=zeta_initial, min=0.1, max=2.0, step=0.1, description='Damping Ratio')

# Create global variables to store the clicked point's coordinates
clicked_x = None
clicked_y = None

# Function to update and plot the step response
def update_plot(zeta):
    den_updated = [1, 2*zeta, 1]
    sys_updated = ctrl.TransferFunction(num, den_updated)
    time, response = ctrl.step_response(sys_updated)
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(time, response)
    ax.set_xlabel('Time')
    ax.set_ylabel('Response')
    ax.set_title(f'Step Response (Damping Ratio = {zeta})')
    ax.grid(True)
    
    # Function to be called when the plot is clicked
    def onclick(event):
        # Display the x and y coordinates of the clicked point
        ax.annotate(f'({event.xdata:.2f}, {event.ydata:.2f})', 
                    xy=(event.xdata, event.ydata), 
                    xytext=(event.xdata+0.1, event.ydata+0.1),
                    arrowprops=dict(facecolor='black', shrink=0.05))
        fig.canvas.draw()

    # Connect the click event to the handler function
    fig.canvas.mpl_connect('button_press_event', onclick)
    
    plt.show()


# Connect the slider to the update function
widgets.interactive(update_plot, zeta=zeta_slider)


In [ ]:
# Analyzing the transient performance

zeta = zeta_slider.value

# Estimations

tc = 1 / (zeta * w) # Time constant
st_5p = 3 * tc      # 5% settling time
st_2p = 4 * tc      # 5% settling time
# Overshoot %
os_p = 100 * np.e**( -(np.pi*zeta) / (np.sqrt(1 - zeta**2)) )

print(f'''
Time Constant: {tc}
5% Settling Time: {st_5p}
2% Settling Time: {st_2p}
Overshoot %: {os_p}
''')

# Some notes
#  zeta < 1  -> Underdamped response
#  zeta == 1 -> Critically damped response
#  zeta > 1  -> Overdamped response
# If you want to test these, I recommend a zeta of
# 0.9 to observe underdamped, 1 to observe critically damped
# and 1.10 to observe overdamped.

In [ ]:
# Gathering transient performance data based on damping ratio
def transient_analysis(zeta, w=1, k=1):
    tc = 1 / (zeta * w) # Time constant
    st_5p = 3 * tc      # 5% settling time
    st_2p = 4 * tc      # 5% settling time
    
    # Overshoot %
    if 1 - zeta**2 <= 0:
        os_p = 0
    else:
        os_p = 100 * np.e**( -(np.pi*zeta) / (np.sqrt(1 - zeta**2)) )
    return (tc, st_5p, st_2p, os_p)

headers = ['Damping Ratio', 'Time Constant','5% Settling Time', '2% Settling Time', 'Overshoot %']
data = [[i]+list(transient_analysis(i)) for i in np.arange(0.01, 2.1, 0.01)]

with open('transient_analysis_out.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(headers)
    writer.writerows(data)



In [ ]:
# Using CSV reader here isn't necessary since we already have
# this data, but it is still useful in case we generate different
# data later that is separate from this notebook

zeta_vals = []
tc_vals = []
st_5p_vals = []
st_2p_vals = []
os_p_vals = []

with open('transient_analysis_out.csv', mode='r', newline='') as csv_file:
    df = pd.read_csv(csv_file)
    zeta_vals = df['Damping Ratio']
    tc_vals = df['Time Constant']
    st_5p_vals = df['5% Settling Time']
    st_2p_vals = df['2% Settling Time']
    os_p_vals = df['Overshoot %']

plt.figure()
plt.plot(zeta_vals, tc_vals)
plt.xlabel('Damping Ratio')
plt.ylabel('Time constant (s)')
plt.show

plt.figure()
plt.plot(zeta_vals, st_5p_vals)
plt.xlabel('Damping Ratio')
plt.ylabel('5% Settling time (s)')
plt.show

plt.figure()
plt.plot(zeta_vals, st_2p_vals)
plt.xlabel('Damping Ratio')
plt.ylabel('2% Settling time')
plt.show

plt.figure()
plt.plot(zeta_vals, os_p_vals)
plt.xlabel('Damping Ratio')
plt.ylabel('Overshoot %')
plt.show
